In [4]:
import numpy as np
import os, random, glob, cv2
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.utils import to_categorical


C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages

In [5]:
label_path = 'D:/DATA/data_2020-05-21_knu_deid/face_angle_class_labels.csv'
df = pd.read_csv(label_path, delimiter=',')
Angle_label = df['Angle Class Model'].tolist() 
ff_img_list = df['ff_path'].tolist() 
fl_img_list = df['fl_path'].tolist() 

In [6]:
data_path = 'D:/DATA/data_2020-05-21_knu_deid/data_2020-05-21_knu_deid/'

In [7]:
ff_img_list = [f'{data_path}{x}' for x in ff_img_list]
fl_img_list = [f'{data_path}{x}' for x in fl_img_list]

In [11]:
img_shape = (256, 256)

ff_img_all = []
fl_img_all = []
for ff_img_path ,fl_img_path in zip(ff_img_list, fl_img_list):
    ff_img = cv2.resize(cv2.cvtColor(cv2.imread(ff_img_path), cv2.COLOR_BGR2RGB), img_shape)
    fl_img = cv2.resize(cv2.cvtColor(cv2.imread(fl_img_path), cv2.COLOR_BGR2RGB), img_shape)
    ff_img_all.append(ff_img)
    fl_img_all.append(fl_img)
ff_img_all = np.array(ff_img_all)/255. 
fl_img_all = np.array(fl_img_all)/255.

np.save('ff_img_all.npy', ff_img_all)
np.save('fl_img_all.npy', fl_img_all)

KeyboardInterrupt: 

In [10]:
def data_generator(ff_img_all, fl_img_all, Angle_label, img_shape, batch_size):
    idx = 0
    idx_list = list(range(len(ff_img_list)))
    random.shuffle(idx_list)
    angle_label_cate = to_categorical(np.array(Angle_label) -1)
    
    while 1:        
        if idx > len(idx_list) - batch_size:
            tmp_list = idx_list[idx:]
            idx = 0
        else:
            tmp_list = idx_list[idx:idx+batch_size] # end of list batch 
            idx = idx + batch_size
#         print(tmp_list)    
        batch_ff_list = ff_img_all[tmp_list]
        batch_fl_list = fl_img_all[tmp_list]
        batch_labels = angle_label_cate[tmp_list, ...]
                
        yield [batch_ff_list, batch_fl_list], batch_labels

In [106]:
def data_generator(ff_img_list, fl_img_list, Angle_label, img_shape, batch_size):
    idx = 0
    idx_list = list(range(len(ff_img_list)))
    random.shuffle(idx_list)
    angle_label_cate = to_categorical(np.array(Angle_label) -1)
    
    while 1:        
        if idx > len(idx_list) - batch_size:
            tmp_list = idx_list[idx:]
            idx = 0
        else:
            tmp_list = idx_list[idx:idx+batch_size] # end of list batch 
            idx = idx + batch_size
#         print(tmp_list)    
        batch_ff_list = np.array(ff_img_list)[tmp_list]
        batch_fl_list = np.array(fl_img_list)[tmp_list]
        batch_labels = angle_label_cate[tmp_list, ...]
        
        batch_ff_img = []
        batch_fl_img = []
        for ff_img_path ,fl_img_path in zip(batch_ff_list, batch_ff_list):
            ff_img = cv2.resize(cv2.cvtColor(cv2.imread(ff_img_path), cv2.COLOR_BGR2RGB), img_shape)
            fl_img = cv2.resize(cv2.cvtColor(cv2.imread(fl_img_path), cv2.COLOR_BGR2RGB), img_shape)
            batch_ff_img.append(ff_img)
            batch_fl_img.append(fl_img)
                
        yield [np.array(batch_ff_img)/255., np.array(batch_fl_img)/255.], batch_labels

In [8]:
batch_size = 32
img_shape = (256, 256)
train_gen = data_generator(ff_img_list, fl_img_list, Angle_label, img_shape, batch_size)

In [9]:
img, lab = next(train_gen)

KeyboardInterrupt: 